In [11]:
import pandas as pd
import numpy as np
df = pd.DataFrame()

lista = ["deporte","futbol","baloncesto","karate","padel","tenis"]

for i in lista:
    data = pd.read_csv(i+".txt", sep='\t')
    
    data["consulta"] = i 
    
    df = pd.concat([df,data],ignore_index=True)
df

,BIBLIOGRAFÍA,AUTOR,TÍTULO,FECHA,CRITERIO,BLOQUE,MEDIO,SOPORTE,TEMA,PAÍS,ZONA,TIPOLOGÍA,NOTAS,CONCORDANCIA,consulta
0,"Almeida, Jorge: Cómo cuidar al bebé. El libro ...","Almeida, Jorge",Cómo cuidar al bebé. El libro de mamá. Nueva o...,1975,Primera edición,No ficción,Escrito,Libro,Salud,Argentina,Río de la Plata,NaN,Incluye al final un breve diccionario pediátri...,"una conversación ""entre mayores"". Si el ambien...",deporte
1,"Palou, Inés: Carne apaleada. Barcelona: Círcul...","Palou, Inés",Carne apaleada,1975,Primera edición,Ficción,Escrito,Libro,Novela,España,NaN,NaN,NaN,"Es gracia que dejen practicar alguna afición, ...",deporte
2,"Sintes Pros, Jorge: Los peligros del colestero...","Sintes Pros, Jorge",Los peligros del colesterol,1975,Primera edición,No ficción,Escrito,Libro,Salud,España,NaN,NaN,El autor no acentúa las mayúsculas,"lo sabe mejor que nunca, pues su vitalidad fís...",deporte
3,"Sintes Pros, Jorge: Los peligros del colestero...","Sintes Pros, Jorge",Los peligros del colesterol,1975,Primera edición,No ficción,Escrito,Libro,Salud,España,NaN,NaN,El autor no acentúa las mayúsculas,La práctica de deportes variados es igualmente...,deporte
4,"Sintes Pros, Jorge: Los peligros del colestero...","Sintes Pros, Jorge",Los peligros del colesterol,1975,Primera edición,No ficción,Escrito,Libro,Salud,España,NaN,NaN,El autor no acentúa las mayúsculas,Pero hay que actuar con cautela. La cultura fí...,deporte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4158,"Anson, Luis María: Don Juan. Barcelona: Plaza ...","Anson, Luis María",Don Juan,1994,Fecha de escritura,No ficción,Escrito,Libro,"Ciencias sociales, creencias y pensamiento",España,NaN,NaN,NaN,"Ama los deportes del mar, pero también el fútb...",tenis
4159,"Anson, Luis María: Don Juan. Barcelona: Plaza ...","Anson, Luis María",Don Juan,1994,Fecha de escritura,No ficción,Escrito,Libro,"Ciencias sociales, creencias y pensamiento",España,NaN,NaN,NaN,ha creído Vuestra Majestad que es Vuestra Maje...,tenis
4160,"Vallejo, Fernando: La virgen de los sicarios. ...","Vallejo, Fernando",La virgen de los sicarios,1994,Primera edición,Ficción,Escrito,Libro,Novela,Colombia,Caribe continental,NaN,NaN,"no lo sabe vaya tomando nota, que cristiano co...",tenis
4161,"Vallejo, Fernando: La virgen de los sicarios. ...","Vallejo, Fernando",La virgen de los sicarios,1994,Primera edición,Ficción,Escrito,Libro,Novela,Colombia,Caribe continental,NaN,NaN,de las pandillas que se estaba decidiendo la o...,tenis


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer


df_copy = df[["TÍTULO", "consulta"]].copy()

mlb = MultiLabelBinarizer()

# Aplicamos el One-Hot Encoding a la columna 'consulta'
palabra_encoded = mlb.fit_transform(df_copy['consulta'].apply(lambda x: [x]))

# Creamos un DataFrame con las palabras como columnas
df_encoded = pd.DataFrame(palabra_encoded, columns=mlb.classes_)

# Concatenamos el DataFrame original con el DataFrame codificado
result = pd.concat([df_copy, df_encoded], axis=1)

# Eliminamos la columna original 'consulta'
result = result.drop('consulta', axis=1)

# Agrupamos por 'titulo' y sumamos las filas con el mismo título
result = result.groupby('TÍTULO').sum().reset_index()



result

for titulo in df["TÍTULO"].unique():
    print(titulo)

Cómo cuidar al bebé. El libro de mamá. Nueva orientación médico-psicológica
Carne apaleada
Los peligros del colesterol
El Comercio
El deporte en la sociedad actual
Funeral por Francia
La gangrena
Infancia es destino
Administración, dirección y supervisión de escuelas
Culminación de Montoya
Cómo cuidar al niño. El libro de mamá
Nada como el piso 16
¡Viva Sandino! Sandino debe nacer...
El Tiempo
¿Por qué corres, Ulises?
Vine, vi... y mejor me fui
Congelar en casa
Descargo de conciencia (1930-1960)
La educación por la danza
Luz de la memoria
El País
La verdadera realidad peruana
La piel del limón
Del buen salvaje al buen revolucionario. Mitos y realidades de América Latina
El beso de la mujer araña
Familia y personalidad
Guatemala: El terremoto de los pobres
La novela criolla en las Antillas
Cuatro idiomas para un Estado. El castellano y los conflictos lingüísticos en la España periférica
Autobiografía de Federico Sánchez
Leyendo historietas. Estilos y sentidos en un &#34;arte menor&#34;


In [40]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import decomposition

vectorizer = CountVectorizer(stop_words='english')


count_vectorizer = CountVectorizer(stop_words='english')

# Ajustar el vectorizador a tus datos
count_data = count_vectorizer.fit_transform(result['TÍTULO'])

# Obtener el vocabulario después del ajuste
vocab = np.array(count_vectorizer.get_feature_names_out())


#vocab = np.array(vectorizer.get_feature_names_out())


tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(result['TÍTULO'])

num_topics = 10

# Aplicar NMF
nmf_model = decomposition.NMF(n_components=num_topics, random_state=50)
W = nmf_model.fit_transform(tfidf)
H = nmf_model.components_

def show_topics(a):
    num_top_words = 1
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

show_topics(H)



['arráncame',
 'américa',
 'aires',
 'asombro',
 'arte',
 'anfitrión',
 'caimito',
 '1930',
 'buenas',
 'aleja']

Los tópicos no tienen mucho que ver, ya que no podemos aplicar tf-id ya que no tenemos el cuerpo de los documentos, solo se toma en cuenta el título, y no les genera el peso a las palabras por la importancia en el tema, sino que todas las toma por igual y los títulos de los libros tampoco tienen mucha relación ya que los dataset de CREA, solo priorizaban que la palabra apareciera, no que tuviera importancia en el documento.